1. Setup

In [2]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D, BatchNormalization, Add
from tensorflow.keras.optimizers import RMSprop, SGD, Adam

In [3]:
# check GPU status
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
# helpers
def count_files(root_path):
    print('count files in:' + root_path)
    assert os.path.exists(root_path)
    total_files = 0
    item_list = os.listdir(root_path)
    if len(item_list) == 0:
        return 0
    for item in item_list:
        next_path = os.path.join(root_path, item)
        if os.path.isfile(next_path):
            total_files += 1
        else:
            total_files += count_files(next_path)

    return total_files


2. Handling inputs

In [ ]:
# Some important directories
DATA_DIR = os.path.join(".", "data")

TEST_DATA_DIR = os.path.join(DATA_DIR, "test")
TRAIN_DATA_DIR = os.path.join(DATA_DIR, "train")

BATCH_SIZE = 128
VALIDATION_RATE = 0.2
IMAGE_SIZE = 48

# count train data number
NUM_TRAIN_DATA = count_files(TRAIN_DATA_DIR)

# load the dataset
# Generate batches of tensor image data with real-time data augmentation
# rescale to convert the pixels in range [0,255] to range [0,1]. This process is Normalizing.
datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=VALIDATION_RATE
)

# Validation data
dategen_valid = ImageDataGenerator(
    rescale=1. / 255,
    validation_split=VALIDATION_RATE
)

# rescale to convert the pixels in range [0,255] to range [0,1]. This process is Normalizing.
datagen_test = ImageDataGenerator(
    rescale=1. / 255,
)